### Домашнее задание 5 - 10 баллов

В этом задании вам предстоит дообучить трансформерную модель для задачи классификации с помощью различных техник и сравнить их между собой.

Датасет: [dair-ai/emotion](https://huggingface.co/datasets/dair-ai/emotion)

Модель: [google-bert/bert-base-uncased](https://huggingface.co/google-bert/bert-base-uncased) (если хочется, можно заменить на что-то более интересное)

1. Скачайте датасет и модель. Измерьте базовые метрики классификации перед началом экспериментов.

**NB!** Для всех типов дообучения замерьте :
- качество классификации на выходе
- время дообучения
- количество параметров для обучения
- потребление ресурсов (не нужно заморачиваться с профайлингом - можно просто посмотреть в `nvidia-smi` или `torch.cuda.memory_allocated`)

2. Обучите модель в режиме full finetuning - **1 балл**
3. Обучите модель в режиме linear probing - реализуйте кастомную классификационную голову и обучайте только ее. Не забудьте описать, чем обусловлено устройство головы, как вы пришли к такой архитектуре - **2 балла**
4. Обучите модель в режиме PEFT с использованием [prompt tuning или prefix tuning](https://ericwiener.github.io/ai-notes/AI-Notes/Large-Language-Models/Prompt-Tuning-and-Prefix-Tuning). При выборе метода напишите пару слов, почему решили остановиться именно на этом методе - **2 балла**
4. Обучите модель в режиме PEFT с использованием LoRA. Попробуйте подобрать оптимальный ранг - `r`, при желании поэкспериментируйте с остальными гиперпараметрами. Опишите, чем обусловлена ваша финальная конфигурация - **2 балла**

5. Соберите все результаты отдельных замеров в таблицу и сделайте выводы о вычислительной сложности методов, итоговом качестве и прочих наблюдаемых свойствах моделей - **1 балл**

**Задание выполнено в Google Colab**

In [ ]:
!pip install evaluate

In [3]:
import torch
import numpy as np
import random
import evaluate
import time
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, PromptTuningConfig
from transformers import DataCollatorWithPadding

In [4]:
# Обеспечим воспроизводимость
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**1. Загрузка данных и модели**

In [ ]:
dataset = load_dataset("dair-ai/emotion")
label_list = dataset["train"].features["label"].names
num_labels = len(label_list)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], truncation=True)

dataset = dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
pd.Series(dataset["train"]["label"]).value_counts()

,count
1,5362
0,4666
3,2159
4,1937
2,1304
5,572


В коллабе есть ограничение на время использование GPU, поэтому не будем трогать количество эпох и learning rate. Будем смотреть сходимость за фиксированное количество эпох. А для оценки метрик в конце используется лучшая модель среди всех эпох по минимальному валидационному лоссу

In [7]:
args = TrainingArguments(
    output_dir="/content/hw_5",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    learning_rate=1e-4,
    seed=SEED,
    load_best_model_at_end=True,
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "Accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "Recall": recall_metric.compute(predictions=predictions, references=labels, average="macro", zero_division=0)[
            "recall"
        ],
        "Precision": precision_metric.compute(
            predictions=predictions, references=labels, average="macro", zero_division=0
        )["precision"],
        "F1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [10]:
eval_results = trainer.evaluate(dataset["test"])
for metric, value in eval_results.items():
    print(f"{metric}: {value:.4f}")
print(f"Trainable parameters: {(sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6):.2f}, M")
print(f"CUDA memory used: {(torch.cuda.memory_allocated() / 1e6):.2f}MB")

eval_loss: 1.6840
eval_model_preparation_time: 0.0030
eval_Accuracy: 0.1510
eval_Recall: 0.1619
eval_Precision: 0.1449
eval_F1: 0.0729
eval_runtime: 5.6283
eval_samples_per_second: 355.3460
eval_steps_per_second: 5.6860
Trainable parameters: 109.49, M
CUDA memory used: 447.61MB


In [13]:
def train_and_show_results(trainer):
    start = time.time()
    trainer.train()
    end = time.time()

    eval_results = trainer.evaluate(dataset["test"])
    for metric, value in eval_results.items():
        print(f"{metric}: {value:.4f}")
    print("Training time:", round(end - start, 2), "sec")
    print(f"Trainable parameters: {(sum(p.numel() for p in trainer.model.parameters() if p.requires_grad) / 1e6):.2f}M")
    print(f"CUDA memory used: {(torch.cuda.memory_allocated() / 1e6):.2f}MB")

**2. Full finetuning**

In [14]:
train_and_show_results(trainer)

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Recall,Precision,F1
1,0.303500,0.241592,0.003000,0.920000,0.913736,0.880857,0.894471
2,0.216900,0.210723,0.003000,0.933500,0.892222,0.931130,0.907770
3,0.168100,0.233989,0.003000,0.926500,0.904639,0.905898,0.901467
4,0.141400,0.267452,0.003000,0.930500,0.903833,0.903611,0.903173
5,0.105300,0.370138,0.003000,0.924000,0.910869,0.889622,0.898964
6,0.073900,0.381330,0.003000,0.931000,0.899350,0.908406,0.903760
7,0.039000,0.425138,0.003000,0.930500,0.897094,0.913173,0.904842
8,0.027300,0.393160,0.003000,0.933500,0.907258,0.904395,0.905700
9,0.016600,0.468375,0.003000,0.933500,0.911238,0.905388,0.907803
10,0.010400,0.482386,0.003000,0.932000,0.907905,0.903346,0.905400


eval_loss: 0.2020
eval_model_preparation_time: 0.0030
eval_Accuracy: 0.9265
eval_Recall: 0.8639
eval_Precision: 0.9136
eval_F1: 0.8830
eval_runtime: 6.5028
eval_samples_per_second: 307.5590
eval_steps_per_second: 4.9210
epoch: 10.0000
Training time: 1783.31 sec
Trainable parameters: 109.49M
CUDA memory used: 1333.99MB


**3. Linear probing**

Загружаем модель и замораживаем Encoder, чтобы обучалась только голова. Здесь используется:
- CLS токен → Dropout → Dense → Softmax

Это простой и эффективный классификатор, не требовательный по ресурсам + класс уже реализован в transformers


In [15]:
lp_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

for param in lp_model.bert.parameters():
    param.requires_grad = False

lp_model = lp_model.to(device)

trainer = Trainer(
    model=lp_model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_and_show_results(trainer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,1.560000,1.546992,0.427500,0.223030,0.140630,0.172138
2,1.532200,1.519435,0.427500,0.218589,0.143581,0.168333
3,1.516400,1.507072,0.450500,0.231203,0.151246,0.178556
4,1.500000,1.500926,0.454000,0.242405,0.153779,0.186170
5,1.492600,1.478775,0.468500,0.242642,0.155058,0.187503
6,1.483100,1.481191,0.465500,0.248182,0.156873,0.190619
7,1.480500,1.468469,0.474500,0.246809,0.156574,0.190684
8,1.473000,1.465585,0.477000,0.249848,0.156922,0.192717
9,1.476800,1.464318,0.478500,0.250227,0.157362,0.193068
10,1.474800,1.462060,0.480500,0.251108,0.158095,0.193804


eval_loss: 1.4335
eval_Accuracy: 0.4640
eval_Recall: 0.2389
eval_Precision: 0.1531
eval_F1: 0.1860
eval_runtime: 6.5645
eval_samples_per_second: 304.6670
eval_steps_per_second: 4.8750
epoch: 10.0000
Training time: 600.46 sec
Trainable parameters: 0.00M
CUDA memory used: 895.94MB


**4. Prompt/Prefix tuning**

Метод **Prefix tuning** звучит интереснее, так как в нем добавляются обучаемые векторы прямо в attention, а не просто в начало текста. А значит влияние этих токенов по логике должно быть сильнее, что может быть довольно полезно в задачах, подобных этой. Такой подход немного тяжелее по памяти + лучше работает на моделях типа BERT.

Однако я буду использовать **Prompt Tuning**, так как он позволяет дообучать небольшое количество виртуальных токенов, которые добавляются в начало входа, а это крутой способ, особенно если нет возможности гонять всю модель. Дополнительной фичей является то, что его легко внедрить через готовые инструменты и он не требует изменений внутри самой модели.

In [17]:
pt_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the emotion in this sentence.",
    num_virtual_tokens=20,
    tokenizer_name_or_path=model_name,
)

pt_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
pt_model = get_peft_model(pt_model, pt_config)
pt_model.print_trainable_parameters()
pt_model.to(device)

trainer = Trainer(
    model=pt_model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_and_show_results(trainer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 15,360 || all params: 109,502,214 || trainable%: 0.0140


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,1.611700,1.591554,0.351500,0.166496,0.100284,0.087268
2,1.597200,1.577233,0.358500,0.172263,0.119499,0.106427
3,1.589200,1.569896,0.384500,0.188286,0.147094,0.131974
4,1.578100,1.563704,0.390500,0.202263,0.128662,0.156108
5,1.574900,1.564147,0.378000,0.182689,0.163678,0.118418
6,1.572600,1.553932,0.422000,0.209953,0.165296,0.157326
7,1.573300,1.553407,0.416000,0.205985,0.171795,0.152590
8,1.561900,1.547219,0.425500,0.212074,0.168752,0.160088
9,1.568400,1.543463,0.433500,0.217055,0.168218,0.165598
10,1.565900,1.545428,0.427000,0.212917,0.170859,0.161135


eval_loss: 1.5239
eval_Accuracy: 0.4235
eval_Recall: 0.2109
eval_Precision: 0.1702
eval_F1: 0.1589
eval_runtime: 9.2131
eval_samples_per_second: 217.0830
eval_steps_per_second: 3.4730
epoch: 10.0000
Training time: 1438.34 sec
Trainable parameters: 0.02M
CUDA memory used: 1334.92MB


**5. LoRA**

In [18]:
for r in [2, 4, 8, 16, 32]:
    print("R = ", r)
    lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=r)

    lora_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    lora_model = get_peft_model(lora_model, lora_config)
    lora_model.print_trainable_parameters()

    lora_model.to(device)

    trainer = Trainer(
        model=lora_model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    train_and_show_results(trainer)
    print()

R =  2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 78,342 || all params: 109,565,196 || trainable%: 0.0715


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,1.022900,0.831250,0.692500,0.486546,0.564912,0.463246
2,0.670900,0.568593,0.800000,0.676687,0.813248,0.706475
3,0.501000,0.406667,0.864500,0.796574,0.860621,0.817945
4,0.414300,0.363120,0.887000,0.840503,0.872995,0.853895
5,0.371400,0.283621,0.907500,0.867005,0.893234,0.878671
6,0.334400,0.255554,0.915000,0.875825,0.896806,0.885486
7,0.292100,0.245937,0.922000,0.889205,0.900985,0.894240
8,0.286700,0.237090,0.921500,0.901226,0.893671,0.896931
9,0.272400,0.234125,0.920500,0.888167,0.894853,0.891086
10,0.278900,0.230692,0.923500,0.894152,0.899698,0.896359


eval_loss: 0.2305
eval_Accuracy: 0.9160
eval_Recall: 0.8553
eval_Precision: 0.8786
eval_F1: 0.8642
eval_runtime: 6.9214
eval_samples_per_second: 288.9590
eval_steps_per_second: 4.6230
epoch: 10.0000
Training time: 1039.32 sec
Trainable parameters: 0.08M
CUDA memory used: 1786.06MB

R =  4


...

eval_loss: 0.2205
eval_Accuracy: 0.9260
eval_Recall: 0.8797
eval_Precision: 0.8982
eval_F1: 0.8868
eval_runtime: 6.8845
eval_samples_per_second: 290.5080
eval_steps_per_second: 4.6480
epoch: 10.0000
Training time: 1050.12 sec
Trainable parameters: 0.59M
CUDA memory used: 1791.24MB

R =  32


No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 1,184,262 || all params: 110,671,116 || trainable%: 1.0701


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.993100,0.781931,0.711500,0.530398,0.592225,0.522438
2,0.603800,0.506970,0.823000,0.715156,0.824947,0.739580
3,0.447100,0.352985,0.880000,0.812128,0.870329,0.831242
4,0.364500,0.302454,0.899000,0.850782,0.889358,0.865788
5,0.320100,0.247632,0.915500,0.879062,0.890407,0.884107
6,0.283400,0.220870,0.925000,0.889054,0.904471,0.896156
7,0.250300,0.219385,0.928000,0.889245,0.914645,0.899807
8,0.245000,0.219204,0.930000,0.903059,0.903367,0.902914
9,0.239800,0.215959,0.928000,0.895047,0.904061,0.898474
10,0.228000,0.213497,0.930000,0.898098,0.907337,0.901774


eval_loss: 0.2120
eval_Accuracy: 0.9270
eval_Recall: 0.8742
eval_Precision: 0.9093
eval_F1: 0.8877
eval_runtime: 6.8926
eval_samples_per_second: 290.1640
eval_steps_per_second: 4.6430
epoch: 10.0000
Training time: 1056.81 sec
Trainable parameters: 1.18M
CUDA memory used: 1797.50MB



In [19]:
data = [
    {
        "R": 2,
        "Trainable Params (M)": 0.08,
        "Trainable %": 0.0715,
        "Accuracy": 0.9160,
        "Recall": 0.8553,
        "Precision": 0.8786,
        "F1": 0.8642,
        "Training Time (s)": 1039.32,
        "Eval Time (s)": 6.9214,
        "CUDA Memory (MB)": 1786.06,
    },
    {
        "R": 4,
        "Trainable Params (M)": 0.15,
        "Trainable %": 0.1387,
        "Accuracy": 0.9220,
        "Recall": 0.8743,
        "Precision": 0.8899,
        "F1": 0.8796,
        "Training Time (s)": 1044.36,
        "Eval Time (s)": 6.9077,
        "CUDA Memory (MB)": 1784.36,
    },
    {
        "R": 8,
        "Trainable Params (M)": 0.30,
        "Trainable %": 0.2728,
        "Accuracy": 0.9235,
        "Recall": 0.8700,
        "Precision": 0.8945,
        "F1": 0.8797,
        "Training Time (s)": 1044.96,
        "Eval Time (s)": 6.9332,
        "CUDA Memory (MB)": 1785.83,
    },
    {
        "R": 16,
        "Trainable Params (M)": 0.59,
        "Trainable %": 0.5400,
        "Accuracy": 0.9260,
        "Recall": 0.8797,
        "Precision": 0.8982,
        "F1":0.8868,
        "Training Time (s)": 1050.12,
        "Eval Time (s)": 6.8845,
        "CUDA Memory (MB)": 1791.24,
    },
    {
        "R": 32,
        "Trainable Params (M)": 1.18,
        "Trainable %": 1.0701,
        "Accuracy": 0.9270,
        "Recall": 0.8742,
        "Precision": 0.9093,
        "F1": 0.8877,
        "Training Time (s)": 1056.81,
        "Eval Time (s)": 6.8926,
        "CUDA Memory (MB)": 1797.50,
    },
]

df_lora = pd.DataFrame(data)
df_lora = df_lora.style.highlight_max(subset=["Accuracy", "Recall", "Precision", "F1"], color="green")
df_lora

,R,Trainable Params (M),Trainable %,Accuracy,Recall,Precision,F1,Training Time (s),Eval Time (s),CUDA Memory (MB)
0,2,0.080000,0.071500,0.916000,0.855300,0.878600,0.864200,1039.320000,6.921400,1786.060000
1,4,0.150000,0.138700,0.922000,0.874300,0.889900,0.879600,1044.360000,6.907700,1784.360000
2,8,0.300000,0.272800,0.923500,0.870000,0.894500,0.879700,1044.960000,6.933200,1785.830000
3,16,0.590000,0.540000,0.926000,0.879700,0.898200,0.886800,1050.120000,6.884500,1791.240000
4,32,1.180000,1.070100,0.927000,0.874200,0.909300,0.887700,1056.810000,6.892600,1797.500000


Мы видим, что качество постепенно растёт с увеличением r, но незначительно. При R = 32 достигается лучший F1-score = 0.8877, но прирост незначителен по сравнению с 16. То есть в целом, можно было его и не проверять. Использование GPU памяти почти не растёт, что делает LoRA очень экономичным. Даже при R = 32 обучается только 1.07% параметров модели (1.18M из 110M)

Вывод такой:
- если намважна эффективность - оптимальным можно считать R = 8 или R = 16
- если хотим легкий бейзлайн - смело берем R = 2



**6. Итоговое сравнение**

In [22]:
data = {
    "method": ["no finetuning", "finetuning", "linear probing", "prompt tuning", "LoRA_R32"],
    "Accuracy": [0.1510, 0.9265, 0.4640, 0.4235, 0.9270],
    "Recall": [0.1619, 0.8639, 0.2389, 0.2109, 0.8742],
    "Precision": [0.1449, 0.9136, 0.1531, 0.1702, 0.9093],
    "F1-score": [0.0729, 0.8830, 0.1860, 0.1589, 0.8877],
    "trainable_params_M": [109.49, 109.49, 0.00, 0.02, 1.18],
    "cuda_memory_MB": [447.61, 1333.99, 895.94, 1334.92, 1797.50],
    "training_time_sec": [None, 1783.31, 600.46, 1438.34, 1056.81],
}

result_df = pd.DataFrame(data)
result_df = (
    result_df.style.format(precision=4)
    .highlight_max(subset=["Accuracy", "Recall", "Precision", "F1-score"], color="green")
    .highlight_min(subset=["trainable_params_M", "cuda_memory_MB", "training_time_sec"], color="green")
)
result_df

,method,Accuracy,Recall,Precision,F1-score,trainable_params_M,cuda_memory_MB,training_time_sec
0,no finetuning,0.1510,0.1619,0.1449,0.0729,109.4900,447.6100,nan
1,finetuning,0.9265,0.8639,0.9136,0.8830,109.4900,1333.9900,1783.3100
2,linear probing,0.4640,0.2389,0.1531,0.1860,0.0000,895.9400,600.4600
3,prompt tuning,0.4235,0.2109,0.1702,0.1589,0.0200,1334.9200,1438.3400
4,LoRA_R32,0.9270,0.8742,0.9093,0.8877,1.1800,1797.5000,1056.8100


**7. Выводы**

- Лучшие показатели у нас вышли на LoRA_R32 - выигрываем по Accuracy, Recall и F1 мере. Немного проигрываем по Precision файнтюнингу, а по времени обучения на втором месте, если не считать no finetuning метод. Единственное, где мы сильно проигрываем - это по используемой памяти.

- Fine-tuning тоже неплохой, поставил бы его на второе место, но он ну очень долго обучался, почти в 2 раза дольше Лоры.

- Prompt tuning и Linear probing - плохенькие по всем показателям, только промпт тюнинг ещё и обучался долго почему-то.

- Ну и конечно, no finetuning дал наихудший результат, разве что по памяти всех выиграл, но он и так понятно почему :)
